In [ ]:
!pip install --quiet -U langgraph langchain langchain-groq tavily-python langchain-community langchainhub

In [ ]:
import os
import getpass

os.environ["TAVILY_API_KEY"]=getpass.getpass("TAVILYAPIKEY")
os.environ["GROQ_API_KEY"]=getpass.getpass("GROQAPIKEY")

TAVILYAPIKEY··········
GROQAPIKEY··········


In [ ]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
tools=[TavilySearchResults(max_results=1)]

prompt=hub.pull("hwchase17/openai-functions-agent")

llm=ChatGroq(model="gemma-7b-it")

agent_runnable=create_openai_functions_agent(llm,tools,prompt)

In [ ]:
import operator
from typing import Annotated, TypedDict, Union

from langchain_core.agents import AgentAction,AgentFinish
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  input:str #input string i.e. main task of the user
  chat_history:list[BaseMessage] #chat history --> prev conv alsopassed  as inputs
  agent_outcome:Union[AgentAction,AgentFinish,None] #outcome of teh agent
  #by default None when it starts
  intermediate_steps:Annotated[list[tuple[AgentAction,str]],operator.add] #corresponding steps

In [ ]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor=ToolExecutor(tools) #helper class to run the tools

def invoke_agent(data):
  agent_outcome=agent_runnable.invoke(data)
  return {"agent_outcome":agent_outcome}

def execute_tools(data):
  #Get the recent agent outcome
  agent_action=data["agent_outcome"]
  output=tool_executor.invoke(agent_action)
  return {"intermediate_steps":[(agent_action,str(output))]}

def should_continue(data):
  if isinstance(data["agent_outcome"],AgentFinish): # If agent outcome is agent finish then we return exit string
    return "end"
  else:
    return "continue"

In [ ]:
from langgraph.graph import END, StateGraph, START

workflow=StateGraph(AgentState)

workflow.add_node("agent",invoke_agent)
workflow.add_node("action",execute_tools)

workflow.add_edge(START,"agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue":"action",
        "end":END
    }
)
workflow.add_edge("action","agent")

app=workflow.compile()

In [ ]:
inputs = {"input": "what is the weather in Kolkata, India", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in Kolkata, India'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in Kolkata, India'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in Kolkata, India"}', 'name': 'tavily_search_results_json'}}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 1002, 'total_tokens': 1090, 'completion_time': 0.100125056, 'prompt_time': 0.604120906, 'queue_time': None, 'total_time': 0.704245962}, 'model_name': 'gemma-7b-it', 'system_fingerprint': 'fp_7d8efeb0b1', 'finish_reason': 'function_call', 'logprobs': None}, id='run-55c30b62-e3b0-4b33-aae9-fb0299939ddc-0', usage_metadata={'input_tokens': 1002, 'output_tokens': 88, 'total_tokens': 1090})])}
----
{'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in Kolkata, India'}, lo